In [40]:
import pandas as pd
from IPython.core.display_functions import display
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from scipy import stats
from scipy.stats import boxcox, yeojohnson
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import optuna
from optuna.samplers import CmaEsSampler
from optuna.samplers import RandomSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectFromModel
from optuna.samplers import TPESampler
from sklearn.metrics import f1_score, roc_auc_score
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.utils import resample
from sklearn.decomposition import PCA

In [41]:
from main import read_xlsb

# download data
train_data_path = "./data/Training.xlsb"
train_sheet_name = 'Training'  # Укажите имя листа, который хотите прочитать
train_data = read_xlsb(train_data_path, train_sheet_name)

test_data_path = "./data/Test.xlsb"
test_sheet_name = 'Test'  # Укажите имя листа, который хотите прочитать
test_data = read_xlsb(test_data_path, test_sheet_name)

# создание train test dataframe
train_data_copy = train_data.copy()
test_data_copy = test_data.copy()
X = train_data_copy.drop(["MARKER","ID"],axis = 1)
X_test = test_data_copy.drop(["MARKER","ID"],axis = 1)
y = train_data_copy['MARKER']
y_test = test_data_copy['MARKER']

#создание копий
X_copy = X.copy()
y_copy= y.copy()

In [42]:
# удаление дублирующих строк
train_data_copy = train_data_copy.drop_duplicates()
print(train_data_copy)


            ID         A         B       C       D         E         F     G  \
0          1.0  0.198778  0.099389    0.00  799.90  1.777556  0.888778  13.0   
1          2.0  0.043000  0.021264   49.97  173.03  0.384511  0.190143  13.0   
2          3.0  0.067073  0.067073    0.00  329.90  0.599818  0.599818  13.0   
3          4.0  0.052700  0.052700    0.00  235.65  0.471300  0.471300  13.0   
4          5.0  0.141880  0.141880    0.00  634.45  1.268900  1.268900  13.0   
...        ...       ...       ...     ...     ...       ...       ...   ...   
89729  89730.0  0.027941  0.014844   25.00  474.00  0.557647  0.296250  20.0   
89730  89731.0  0.120017  0.068581  179.96  720.03  1.200050  0.685743  10.0   
89731  89732.0  0.153033  0.122427   29.01  550.99  0.459158  0.367327   3.0   
89732  89733.0  0.107575  0.043030   53.82  215.18  0.537950  0.215180   5.0   
89733  89734.0  0.095750  0.095750   57.48  172.42  0.287367  0.287367   3.0   

          H      I  ...       P        

In [43]:
# кодирование категориальных признаков
def encode_categorical_features(df):
    encoded_df = df.copy()

    # Шаг 1: Выбор категориальных признаков
    categorical_columns = df.select_dtypes(include=['object']).columns

    # Шаг 2: Закодировать категориальные признаки
    for column in categorical_columns:
        unique_values = df[column].nunique()
        if column == 'P':
            # Отдельный метод для столбца 'P'
            encoded_values = {
                '0 Zero': 0,
                '1 One': 1,
                '2 Two': 2,
                '3 Three': 3,
                'More than 3': 4
            }
            encoded_df[column] = df[column].map(encoded_values)
        elif unique_values <= 2:
            # One-hot encoding для признаков с <= 2 уникальными значениями
            encoder = LabelEncoder()
            encoded_df[column] = encoder.fit_transform(df[column])
        else:
            # Count encoding для признаков с > 2 уникальными значениями
            counter = df[column].value_counts().to_dict()
            encoded_df[column] = df[column].map(counter)

    return encoded_df


train_data_copy[X.columns] = encode_categorical_features(train_data_copy[X.columns])
print(train_data_copy)
train_data_copy.to_csv('encoded_train_data.csv')

test_data_copy[X_test.columns] = encode_categorical_features(test_data_copy[X_test.columns])
print(test_data_copy)
test_data_copy.to_csv('encoded_test_data.csv')

            ID         A         B       C       D         E         F     G  \
0          1.0  0.198778  0.099389    0.00  799.90  1.777556  0.888778  13.0   
1          2.0  0.043000  0.021264   49.97  173.03  0.384511  0.190143  13.0   
2          3.0  0.067073  0.067073    0.00  329.90  0.599818  0.599818  13.0   
3          4.0  0.052700  0.052700    0.00  235.65  0.471300  0.471300  13.0   
4          5.0  0.141880  0.141880    0.00  634.45  1.268900  1.268900  13.0   
...        ...       ...       ...     ...     ...       ...       ...   ...   
89729  89730.0  0.027941  0.014844   25.00  474.00  0.557647  0.296250  20.0   
89730  89731.0  0.120017  0.068581  179.96  720.03  1.200050  0.685743  10.0   
89731  89732.0  0.153033  0.122427   29.01  550.99  0.459158  0.367327   3.0   
89732  89733.0  0.107575  0.043030   53.82  215.18  0.537950  0.215180   5.0   
89733  89734.0  0.095750  0.095750   57.48  172.42  0.287367  0.287367   3.0   

          H  I  ...  P      Q  R      S

In [44]:
"""
def normalize_columns(df, small_value=1e-10):
    normalized_df = pd.DataFrame()

    for column in df.columns:
        data = df[column]

        # Применение всех перечисленных методов нормализации
        methods = [
            (np.log1p,),
            (lambda x: 1 / (x + small_value) if (x != 0).all() else x,),
            (np.sqrt,),
            (np.square,),
            (boxcox,),
            (yeojohnson,),
            (boxcox, yeojohnson),
            (np.log1p, boxcox),
            (np.log1p, yeojohnson),
            (np.log1p, boxcox, yeojohnson)
        ]

        # Переменные для выбора наилучшей комбинации методов
        best_methods = None
        best_p_value = 0.0

        for method_group in methods:
            transformed_data = data

            for method in method_group:
                if method in [boxcox, yeojohnson]:
                    if (transformed_data == 0).any():
                        transformed_data, _ = method(transformed_data + small_value)
                    else:
                        transformed_data, _ = method(transformed_data)
                else:
                    transformed_data = method(transformed_data)

            _, p_value = stats.normaltest(transformed_data)
            print("столбец:",column,"метод:",method,"текущий p_value:",p_value)
            if p_value > best_p_value:
                best_p_value = p_value
                best_methods = method_group

        # Применение наилучшей комбинации методов к столбцу
        transformed_data = data

        if best_methods:
            for method in best_methods:
                if method in [boxcox, yeojohnson]:
                    if (transformed_data == 0).any():
                        transformed_data, _ = method(transformed_data + small_value)
                    else:
                        transformed_data, _ = method(transformed_data)
                else:
                    transformed_data = method(transformed_data)

        normalized_df[column] = transformed_data

    return normalized_df

X_copy = normalize_columns(X_copy)
print(X_copy)
"""

'\ndef normalize_columns(df, small_value=1e-10):\n    normalized_df = pd.DataFrame()\n\n    for column in df.columns:\n        data = df[column]\n\n        # Применение всех перечисленных методов нормализации\n        methods = [\n            (np.log1p,),\n            (lambda x: 1 / (x + small_value) if (x != 0).all() else x,),\n            (np.sqrt,),\n            (np.square,),\n            (boxcox,),\n            (yeojohnson,),\n            (boxcox, yeojohnson),\n            (np.log1p, boxcox),\n            (np.log1p, yeojohnson),\n            (np.log1p, boxcox, yeojohnson)\n        ]\n\n        # Переменные для выбора наилучшей комбинации методов\n        best_methods = None\n        best_p_value = 0.0\n\n        for method_group in methods:\n            transformed_data = data\n\n            for method in method_group:\n                if method in [boxcox, yeojohnson]:\n                    if (transformed_data == 0).any():\n                        transformed_data, _ = method(tran

In [45]:
def remove_outliers(train_data, num_columns_threshold=5, ignore_columns=[], remove_class_1_outliers=True):
    print("len train_data input", len(train_data))
    # Выбираем только столбцы, которые не находятся в списке ignore_columns
    columns_to_analyze = [col for col in train_data.columns if col not in ignore_columns]
    df = train_data[columns_to_analyze]

    # Подсчитываем Q1 и Q3 для каждого столбца
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)

    # Рассчитываем межквартильный размах (IQR)
    iqr = q3 - q1

    # Определяем границы интервала для удаления выбросов
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Создаем маску для определения строк с выбросами в каждом столбце
    outliers_mask = ((df < lower_bound) | (df > upper_bound))

    # Подсчитываем количество столбцов с выбросами в каждой строке
    num_outliers = outliers_mask.sum(axis=1)

    # Создаем маску для определения строк с выбросами в 5 и более столбцах
    mask = (num_outliers >= num_columns_threshold)

    # Удаляем строки с выбросами из train_data
    if remove_class_1_outliers:
        mask_to_remove = mask
    else:
        mask_to_remove = mask & (train_data['MARKER'] != 1)
    train_data_cleaned = train_data[~mask_to_remove]

    print("len train_data_cleaned", len(train_data_cleaned))
    # Подсчитываем количество удаленных строк для каждого класса
    num_deleted_class_0 = len(train_data[train_data['MARKER'] == 0]) - len(train_data_cleaned[train_data_cleaned['MARKER'] == 0])
    num_deleted_class_1 = len(train_data[train_data['MARKER'] == 1]) - len(train_data_cleaned[train_data_cleaned['MARKER'] == 1])
    print("Deleted rows from class 0:", num_deleted_class_0)
    print("Deleted rows from class 1:", num_deleted_class_1)

    return train_data_cleaned

# Пример использования функции с удалением строк из класса MARKER = 1
train_data_copy = remove_outliers(train_data_copy, num_columns_threshold=5, ignore_columns=['ID', 'MARKER'], remove_class_1_outliers=True)
print(train_data_copy)
train_data_copy.to_csv('remove_outliers_train_data.csv')


len train_data input 89734
len train_data_cleaned 87350
Deleted rows from class 0: 2366
Deleted rows from class 1: 18
            ID         A         B       C       D         E         F     G  \
0          1.0  0.198778  0.099389    0.00  799.90  1.777556  0.888778  13.0   
1          2.0  0.043000  0.021264   49.97  173.03  0.384511  0.190143  13.0   
2          3.0  0.067073  0.067073    0.00  329.90  0.599818  0.599818  13.0   
3          4.0  0.052700  0.052700    0.00  235.65  0.471300  0.471300  13.0   
4          5.0  0.141880  0.141880    0.00  634.45  1.268900  1.268900  13.0   
...        ...       ...       ...     ...     ...       ...       ...   ...   
89729  89730.0  0.027941  0.014844   25.00  474.00  0.557647  0.296250  20.0   
89730  89731.0  0.120017  0.068581  179.96  720.03  1.200050  0.685743  10.0   
89731  89732.0  0.153033  0.122427   29.01  550.99  0.459158  0.367327   3.0   
89732  89733.0  0.107575  0.043030   53.82  215.18  0.537950  0.215180   5.0   
89

In [46]:
# масштабирование
def robust_scale_dataframe(df, ignore_columns=[]):
    columns_to_scale = [col for col in df.columns if col not in ignore_columns]
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(df[columns_to_scale])

    # Обновляем значения в столбцах, которые нужно масштабировать
    df.loc[:, columns_to_scale] = scaled_data
    return df
"""
train_data_copy = robust_scale_dataframe(train_data_copy,ignore_columns=['ID', 'MARKER'])
print(train_data_copy)
train_data_copy.to_csv('prepar_train_data.csv')

test_data_copy = robust_scale_dataframe(test_data_copy,ignore_columns=['ID', 'MARKER'])
print(test_data_copy)
test_data_copy.to_csv('prepar_test_data.csv')
"""

"\ntrain_data_copy = robust_scale_dataframe(train_data_copy,ignore_columns=['ID', 'MARKER'])\nprint(train_data_copy)\ntrain_data_copy.to_csv('prepar_train_data.csv')\n\ntest_data_copy = robust_scale_dataframe(test_data_copy,ignore_columns=['ID', 'MARKER'])\nprint(test_data_copy)\ntest_data_copy.to_csv('prepar_test_data.csv')\n"

In [47]:
# масштабирование с выбором способа
def scale_dataframe(df, ignore_columns=[], scaler_type='robust'):
    columns_to_scale = [col for col in df.columns if col not in ignore_columns]

    if scaler_type == 'standard':
        scaler = StandardScaler()
    elif scaler_type == 'robust':
        scaler = RobustScaler()
    else:
        raise ValueError("Invalid scaler_type. Use 'standard' or 'robust'.")

    scaled_data = scaler.fit_transform(df[columns_to_scale])

    # Обновляем значения в столбцах, которые нужно масштабировать
    df.loc[:, columns_to_scale] = scaled_data
    return df

# Пример использования с разными способами масштабирования
train_data_copy = scale_dataframe(train_data_copy,ignore_columns=['ID', 'MARKER'],scaler_type= "standard")
print(train_data_copy)
train_data_copy.to_csv('prepar_train_data.csv')

test_data_copy = scale_dataframe(test_data_copy,ignore_columns=['ID', 'MARKER'],scaler_type= "standard")
print(test_data_copy)
test_data_copy.to_csv('prepar_test_data.csv')


            ID         A         B         C         D         E         F  \
0          1.0  0.260126 -0.096338 -0.851553  0.650218  1.024437  0.448462   
1          2.0 -0.829576 -0.893296 -0.330364 -0.843031 -0.719023 -0.837314   
2          3.0 -0.661182 -0.425997 -0.851553 -0.469356 -0.449556 -0.083343   
3          4.0 -0.761722 -0.572614 -0.851553 -0.693866 -0.610402 -0.319869   
4          5.0 -0.137887  0.337116 -0.851553  0.256104  0.387831  1.148043   
...        ...       ...       ...       ...       ...       ...       ...   
89729  89730.0 -0.934916 -0.958787 -0.590802 -0.126099 -0.502335 -0.642033   
89730  89731.0 -0.290827 -0.410611  1.025437  0.459962  0.301662  0.074794   
89731  89732.0 -0.059867  0.138671 -0.548978  0.057296 -0.625598 -0.511223   
89732  89733.0 -0.377859 -0.671258 -0.290208 -0.742627 -0.526987 -0.791235   
89733  89734.0 -0.460577 -0.133459 -0.252035 -0.844484 -0.840603 -0.658382   

              G         H         I  ...         P         Q   

In [48]:
# общий отбор признаков
# Функция для удаления признаков с малым разнообразием значений
def remove_low_variance_features(df, threshold=0.95, ignore_columns=[]):
    num_rows = len(df)
    columns_to_check = [col for col in df.columns if col not in ignore_columns]
    low_variance_columns = [col for col in columns_to_check if (df[col].value_counts() / num_rows).max() > threshold]
    df_filtered = df.drop(columns=low_variance_columns)
    return df_filtered

# Функция для удаления признаков с высокой корреляцией между собой
def remove_high_correlation_features(df, y_df, threshold=0.8, ignore_columns=['ID', 'MARKER']):
    corr_matrix = df.corr()  # Вычисляет матрицу корреляции для всех столбцов в df
    to_drop = []  # Создает пустой список, в который будут добавляться столбцы для удаления

    for column in corr_matrix.columns:  # Проходит по каждому столбцу матрицы корреляции
        if column in ignore_columns:
            continue  # Пропускает столбцы, указанные в ignore_columns

        correlated_columns = corr_matrix.index[corr_matrix[column] > threshold].tolist()  # Находит список столбцов, которые сильно коррелируют с текущим столбцом
        if correlated_columns:  # Если список сильно коррелирующих столбцов не пустой
            max_corr_with_target = max(correlated_columns, key=lambda col: abs(df[column].corr(y_df)))
            # Находит столбец с максимальной абсолютной корреляцией с целевым признаком y_df

            if column != max_corr_with_target and max_corr_with_target not in ignore_columns:  # Добавляем проверку, чтобы игнорировать столбцы из ignore_columns
                to_drop.append(column)  # Добавляет текущий столбец в список для удаления

    df_filtered = df.drop(columns=to_drop)  # Удаляет столбцы из списка to_drop из исходного dataframe df
    return df_filtered  # Возвращает новый dataframe с удаленными столбцами


#Univariate feature selection
"""
def select_features_f_regression(X, y, k=10, ignore_columns=['ID', 'MARKER']):
    selector = SelectKBest(score_func=f_regression, k=k)
    X_new = selector.fit_transform(X, y)
    selected_features = X.columns[selector.get_support()]

    # Создаем новый DataFrame с выбранными признаками
    X_new_df = pd.DataFrame(X_new, columns=selected_features, index=X.index)

    # Восстанавливаем игнорируемые столбцы на своих местах
    for col in ignore_columns:
        X_new_df[col] = X[col]

    return X_new_df, selected_features
"""

def select_features_roc_auc(X, y, k=10, ignore_columns=['ID', 'MARKER']):
    selector = SelectKBest(score_func=f_classif, k=k)
    X_new = selector.fit_transform(X, y)
    selected_features = X.columns[selector.get_support()]

    X_new_df = pd.DataFrame(X_new, columns=selected_features, index=X.index)

    for col in ignore_columns:
        X_new_df[col] = X[col]

    return X_new_df, selected_features

"""
def select_features_pca(X, y, variance_threshold=0.8, ignore_columns=['ID']):
    # Удаляем столбец "ID" из X, если он присутствует
    X = X.drop(columns=ignore_columns, errors='ignore')

    # Создаем модель PCA, указываем, что мы хотим сохранить variance_threshold дисперсии
    pca = PCA(n_components=variance_threshold, svd_solver='full')

    # Применяем PCA на матрице признаков X
    X_pca = pca.fit_transform(X)

    # Создаем новый DataFrame с выбранными компонентами
    selected_features = ['PCA_Component_{}'.format(i) for i in range(X_pca.shape[1])]
    X_pca_df = pd.DataFrame(X_pca, columns=selected_features, index=X.index)

    return X_pca_df, selected_features

"""


train_data_copy = remove_low_variance_features(train_data_copy,threshold=0.85, ignore_columns=['ID', 'MARKER'])
print(train_data_copy)

train_data_copy = remove_high_correlation_features(train_data_copy,y_copy,threshold=0.75,ignore_columns=['ID', 'MARKER'])
print(train_data_copy)

train_data_copy,important_columns = select_features_roc_auc(train_data_copy, train_data_copy["MARKER"], k=17)
print(train_data_copy)

"""
train_data_copy, important_columns = select_features_pca(train_data_copy, train_data_copy["MARKER"])
print(train_data_copy)
"""


            ID         A         B         C         D         E         F  \
0          1.0  0.260126 -0.096338 -0.851553  0.650218  1.024437  0.448462   
1          2.0 -0.829576 -0.893296 -0.330364 -0.843031 -0.719023 -0.837314   
2          3.0 -0.661182 -0.425997 -0.851553 -0.469356 -0.449556 -0.083343   
3          4.0 -0.761722 -0.572614 -0.851553 -0.693866 -0.610402 -0.319869   
4          5.0 -0.137887  0.337116 -0.851553  0.256104  0.387831  1.148043   
...        ...       ...       ...       ...       ...       ...       ...   
89729  89730.0 -0.934916 -0.958787 -0.590802 -0.126099 -0.502335 -0.642033   
89730  89731.0 -0.290827 -0.410611  1.025437  0.459962  0.301662  0.074794   
89731  89732.0 -0.059867  0.138671 -0.548978  0.057296 -0.625598 -0.511223   
89732  89733.0 -0.377859 -0.671258 -0.290208 -0.742627 -0.526987 -0.791235   
89733  89734.0 -0.460577 -0.133459 -0.252035 -0.844484 -0.840603 -0.658382   

              G         H         I  ...         M         N   

D:\PycharmProjects\innowise\venv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


'\ntrain_data_copy, important_columns = select_features_pca(train_data_copy, train_data_copy["MARKER"])\nprint(train_data_copy)\n'

In [49]:
"""
from sklearn.metrics import roc_auc_score

def objective(trial, df, target_column_name, threshold, cv):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    C = trial.suggest_float('C', 1e-2, 1e5, log=True)

    X = df.iloc[:, 1:-1]
    y = df[target_column_name]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    logreg = LogisticRegression(penalty=penalty, C=C, solver='liblinear')
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)
    roc_auc = roc_auc_score(y_test, y_pred)

    return roc_auc

def build_logistic_regression_model(df, target_column_name, threshold=0.1, cv=5):
    sampler = optuna.samplers.RandomSampler()

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(lambda trial: objective(trial, df, target_column_name, threshold, cv), n_trials=50)

    best_params = study.best_params
    best_penalty = best_params['penalty']
    best_C = best_params['C']

    X = df.iloc[:, 1:-1]
    y = df[target_column_name]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    logreg = LogisticRegression(penalty=best_penalty, C=best_C, solver='liblinear')
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)
    roc_auc = roc_auc_score(y_test, y_pred)

    coefficients = logreg.coef_[0]

    feature_names = X_train.columns
    coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
    coefficients_df = coefficients_df.reindex(coefficients_df['Coefficient'].abs().sort_values(ascending=False).index)

    selected_features = coefficients_df[coefficients_df['Coefficient'].abs() > threshold]['Feature'].tolist()

    X_train_selected = X_train[selected_features]

    scores = cross_val_score(logreg, X_train_selected, y_train, cv=cv, error_score='raise')
    average_score = scores.mean()

    final_model = logreg.fit(X_train_selected, y_train)

    return final_model, selected_features, average_score

# Assuming train_data_copy contains your data
final_model, selected_features, average_score = build_logistic_regression_model(train_data_copy.drop('ID', axis=1), target_column_name="MARKER")
print("Selected Features:", selected_features)
print("Average ROC AUC Score:", average_score)
"""

'\nfrom sklearn.metrics import roc_auc_score\n\ndef objective(trial, df, target_column_name, threshold, cv):\n    penalty = trial.suggest_categorical(\'penalty\', [\'l1\', \'l2\'])\n    C = trial.suggest_float(\'C\', 1e-2, 1e5, log=True)\n\n    X = df.iloc[:, 1:-1]\n    y = df[target_column_name]\n\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n    logreg = LogisticRegression(penalty=penalty, C=C, solver=\'liblinear\')\n    logreg.fit(X_train, y_train)\n\n    y_pred = logreg.predict(X_test)\n    roc_auc = roc_auc_score(y_test, y_pred)\n\n    return roc_auc\n\ndef build_logistic_regression_model(df, target_column_name, threshold=0.1, cv=5):\n    sampler = optuna.samplers.RandomSampler()\n\n    study = optuna.create_study(direction=\'maximize\', sampler=sampler)\n    study.optimize(lambda trial: objective(trial, df, target_column_name, threshold, cv), n_trials=50)\n\n    best_params = study.best_params\n    best_penalty = best_params[\'p

In [50]:
"""
# логистическая регрессия 2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def build_logistic_regression_model(df, target_column_name):
    # Выделите признаки и целевую переменную
    X = df.drop(columns=['ID', target_column_name])
    y = df[target_column_name]

    # Разделите данные на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Определите параметры для перебора
    param_grid = {
        'penalty': ['l1', 'l2'],  # Перебор L1 и L2 регуляризации
        'C': np.logspace(-1, 1, 20)  # Перебор параметра регуляризации C
    }

    # Создайте модель логистической регрессии с взвешиванием классов
    logreg = LogisticRegression(class_weight='balanced', solver='liblinear')

    # Создайте объект GridSearchCV для перебора параметров
    grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='f1')

    # Выполните поиск по сетке на обучающем наборе
    grid_search.fit(X_train, y_train)

    # Получите наилучшие параметры и значение метрики f1_score
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print("Best Parameters:", best_params)
    print("Best F1 Score:", best_score)

    # Получите наилучшую модель
    best_model = grid_search.best_estimator_

    # Обучите модель на всем обучающем наборе
    best_model.fit(X_train, y_train)

    # Предсказания для тестового набора
    y_pred = best_model.predict(X_test)

    # Вычислите метрику F1-score на тестовом наборе
    test_f1_score = f1_score(y_test, y_pred)
    print("Test F1 Score:", test_f1_score)

    # Выведите коэффициенты модели
    coefficients = best_model.coef_[0]
    feature_names = X.columns
    coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
    coefficients_df = coefficients_df.reindex(coefficients_df['Coefficient'].abs().sort_values(ascending=False).index)
    print(coefficients_df)

    return best_model, coefficients_df

# Пример использования:
final_model, coefficients = build_logistic_regression_model(train_data_copy, target_column_name="MARKER")
print("Selected Features:", coefficients)
"""

'\n# логистическая регрессия 2\nimport numpy as np\nimport pandas as pd\nfrom sklearn.model_selection import train_test_split, GridSearchCV\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import f1_score\n\ndef build_logistic_regression_model(df, target_column_name):\n    # Выделите признаки и целевую переменную\n    X = df.drop(columns=[\'ID\', target_column_name])\n    y = df[target_column_name]\n\n    # Разделите данные на обучающий и тестовый наборы\n    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n    # Определите параметры для перебора\n    param_grid = {\n        \'penalty\': [\'l1\', \'l2\'],  # Перебор L1 и L2 регуляризации\n        \'C\': np.logspace(-1, 1, 20)  # Перебор параметра регуляризации C\n    }\n\n    # Создайте модель логистической регрессии с взвешиванием классов\n    logreg = LogisticRegression(class_weight=\'balanced\', solver=\'liblinear\')\n\n    # Создайте объект GridSearchCV для переб

In [51]:
# Oversampling

def perform_random_oversampling(df, target_column_name):
    # Выделите признаки и целевую переменную
    """
    X = df.drop(columns=[target_column_name])
    y = df[target_column_name]
    """
    # Разделим данные по значениям целевой переменной
    df_majority = df[df[target_column_name] == 0]
    df_minority = df[df[target_column_name] == 1]

    # Применим Random Over-sampling к классу с меньшим количеством записей
    df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=42)

    # Объединим балансированные классы
    balanced_df = pd.concat([df_majority, df_minority_upsampled])

    return balanced_df

"""
# Примените функцию для создания новой таблицы с балансированными классами
train_data_copy = perform_random_oversampling(train_data_copy, target_column_name="MARKER")
print(train_data_copy)
train_data_copy.to_csv("random_oversampling.csv")
"""

'\n# Примените функцию для создания новой таблицы с балансированными классами\ntrain_data_copy = perform_random_oversampling(train_data_copy, target_column_name="MARKER")\nprint(train_data_copy)\ntrain_data_copy.to_csv("random_oversampling.csv")\n'

In [52]:
# логистическая регрессия 3
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# ...

def build_logistic_regression_model(df, target_column_name):
    # Выделите признаки и целевую переменную
    X = df.drop(columns=['ID', target_column_name])
    y = df[target_column_name]

    # Разделите данные на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Определите параметры для перебора
    param_grid = {
        'penalty': ['l1', 'l2'],  # Перебор L1 и L2 регуляризации
        'C': np.logspace(-4, 4, 30)  # Перебор параметра регуляризации C
    }

    # Создайте модель логистической регрессии
    logreg = LogisticRegression(solver='liblinear', class_weight='balanced')

    # Создайте объект GridSearchCV для перебора параметров
    grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='f1')

    # Выполните поиск по сетке на обучающем наборе
    grid_search.fit(X_train, y_train)

    # Получите наилучшие параметры и значение метрики F1-score
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print("Best Parameters:", best_params)
    print("Best F1 Score:", best_score)

    # Получите наилучшую модель
    best_model = grid_search.best_estimator_

    # Обучите модель на выбранных наиболее важных признаках
    coefficients = best_model.coef_[0]
    feature_names = X.columns
    coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

    # Отберите 10 наиболее важных признаков
    top_features = coefficients_df.iloc[np.abs(coefficients_df['Coefficient']).nlargest(10).index]
    selected_features = top_features['Feature'].tolist()
    X_train_selected = X_train[selected_features]
    X_test_selected = X_test[selected_features]

    # Обучение модели на выбранных признаках
    best_model.fit(X_train_selected, y_train)

    # Предсказания для тестового набора
    y_pred = best_model.predict(X_test_selected)

    # Вычислите метрику F1-score на тестовом наборе
    test_f1_score = f1_score(y_test, y_pred)
    print("Test F1 Score:", test_f1_score)

    return best_model, top_features


def evaluate_with_threshold(model, test_data, target_column_name, selected_features, threshold=0.5):
    # Выделите признаки из test данных
    test_X = test_data[selected_features]

    # Получите вероятности для класса 1
    test_probabilities = model.predict_proba(test_X)[:, 1]

    # Примените порог вероятности для получения предсказаний (0 или 1)
    test_predictions = (test_probabilities > threshold).astype(int)

    # Создайте DataFrame с предсказанными значениями
    predictions_df = pd.DataFrame({'Prediction': test_predictions})

    # Запишите DataFrame в CSV-файл
    predictions_df.to_csv("predictions.csv", index=False)

    target_data = test_data[target_column_name]

    # Вычислим метрику F1-score на test данных
    test_f1 = f1_score(target_data, test_predictions)

    # Вычислим метрику ROC AUC на test данных
    test_roc_auc = roc_auc_score(target_data, test_probabilities)

    return test_predictions, test_f1, test_roc_auc



final_model, coefficients = build_logistic_regression_model(train_data_copy, target_column_name="MARKER")
print("Selected Features:", coefficients)

# Получите список выбранных признаков
selected_features = coefficients['Feature'].tolist()

# Используйте разные значения порога вероятности и смотрите, как изменяются метрики
threshold = 0.965  # Пример значения порога
test_predictions, test_f1_score, test_roc_auc = evaluate_with_threshold(final_model, test_data_copy, target_column_name="MARKER", selected_features=selected_features, threshold=threshold)
print("Test Predictions:", test_predictions)
print("Test F1 Score (Threshold={}):".format(threshold), test_f1_score)
print("Test ROC AUC Score (Threshold={}):".format(threshold), test_roc_auc)





Best Parameters: {'C': 0.008531678524172805, 'penalty': 'l1'}
Best F1 Score: 0.026844778613515898
Test F1 Score: 0.031982942430703626
Selected Features:    Feature  Coefficient
4        H     0.965258
1        D     0.701302
7        L    -0.626044
9        N     0.554523
11       R    -0.446009
10       O    -0.441224
13       W    -0.282629
14       X    -0.211366
8        M    -0.205839
3        G    -0.197778
Test Predictions: [0 1 0 ... 0 0 0]
Test F1 Score (Threshold=0.965): 0.1111111111111111
Test ROC AUC Score (Threshold=0.965): 0.8689360225743283


In [53]:
# случайный лес

def objective(trial, df, target_column_name, cv):
    n_estimators = trial.suggest_int('n_estimators', 50, 750, step= 100)
    max_depth = trial.suggest_int('max_depth',3, 15)
    min_samples_split = trial.suggest_float('min_samples_split', 0.05, 1.0)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.05, 0.5)


    X = df.iloc[:, :-1]
    y = df[target_column_name]

    model = RandomForestClassifier(
    n_estimators=n_estimators,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    class_weight='balanced',  # Добавьте этот параметр
    random_state=42
)

    scores = cross_val_score(model, X, y, cv=cv)
    average_score = scores.mean()

    return average_score

# ...

def build_random_forest_model(df, target_column_name, cv=5, selected_features=None):
    sampler = TPESampler()

    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(lambda trial: objective(trial, df, target_column_name, cv), n_trials=40)

    best_params = study.best_params
    best_n_estimators = best_params['n_estimators']
    best_max_depth = best_params['max_depth']
    best_min_samples_split = best_params['min_samples_split']
    best_min_samples_leaf = best_params['min_samples_leaf']

    X = df.iloc[:, :-1]

    if selected_features is not None:
        print("Selected Features:", selected_features)
        X = X[selected_features]

    y = df[target_column_name]

    model = RandomForestClassifier(
        n_estimators=best_n_estimators,
        max_depth=best_max_depth,
        min_samples_split=best_min_samples_split,
        min_samples_leaf=best_min_samples_leaf,
        class_weight='balanced',  # Добавьте этот параметр
        random_state=42
    )

    model.fit(X, y)

    feature_importances = model.feature_importances_
    feature_names = X.columns
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
    print(feature_importance_df)
    #selected_features = feature_importance_df[feature_importance_df['Importance'] > 0]['Feature'].tolist()
    selected_features = feature_importance_df['Feature'].tolist()
    print(selected_features)
    """
    if len(selected_features) > 0:
        X_selected = X[selected_features]
        scores = cross_val_score(model, X_selected, y, cv=cv)
        average_score = scores.mean()
    else:
        average_score = 0.0  # Произвольное значение для случая, когда нет выбранных признаков
    """
    X_selected = X[selected_features]
    scores = cross_val_score(model, X_selected, y, cv=cv)
    average_score = scores.mean()
    return model, selected_features, average_score

"""
final_model, selected_features, average_score = build_random_forest_model(train_data_copy.drop('ID', axis=1), target_column_name="MARKER",selected_features= selected_features)
print("Selected Features:", selected_features)
print("Average Cross-Validation Score:", average_score)
"""

'\nfinal_model, selected_features, average_score = build_random_forest_model(train_data_copy.drop(\'ID\', axis=1), target_column_name="MARKER",selected_features= selected_features)\nprint("Selected Features:", selected_features)\nprint("Average Cross-Validation Score:", average_score)\n'

In [54]:
# случайный лес 2
def objective(trial, X, y):
    n_estimators = trial.suggest_int('n_estimators', 50, 750, step=100)
    max_depth = trial.suggest_int('max_depth', 3, 25)
    min_samples_split = trial.suggest_float('min_samples_split', 0.05, 1.0)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.05, 0.5)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        class_weight='balanced',
        random_state=42
    )

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=skf, scoring='roc_auc')
    average_score = scores.mean()

    return average_score

def build_and_predict_random_forest(train_data, test_data, target_column_name, selected_features):
    # Выделите признаки и целевую переменную
    X_train = train_data[selected_features]
    y_train = train_data[target_column_name]
    X_test = test_data[selected_features]
    y_test = test_data[target_column_name]

    # Подбор гиперпараметров
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=40)

    best_params = study.best_params
    best_n_estimators = best_params['n_estimators']
    best_max_depth = best_params['max_depth']
    best_min_samples_split = best_params['min_samples_split']
    best_min_samples_leaf = best_params['min_samples_leaf']

    # Создайте и обучите модель случайного леса с наилучшими гиперпараметрами
    model = RandomForestClassifier(
        n_estimators=best_n_estimators,
        max_depth=best_max_depth,
        min_samples_split=best_min_samples_split,
        min_samples_leaf=best_min_samples_leaf,
        class_weight='balanced',
        random_state=42
    )
    model.fit(X_train, y_train)

    # Прогнозирование на тестовых данных
    test_predictions = model.predict(X_test)

    # Вычисление метрик
    roc_auc = roc_auc_score(y_test, test_predictions)
    classification_rep = classification_report(y_test, test_predictions)

    print("ROC AUC:", roc_auc)
    print("Classification Report:\n", classification_rep)

    return model

# Построение и оценка модели случайного леса
model = build_and_predict_random_forest(train_data_copy, test_data_copy, "MARKER", selected_features)


[I 2023-08-18 17:08:34,178] A new study created in memory with name: no-name-b372f5db-3bcf-4f9c-9be5-91475876b044
[I 2023-08-18 17:08:48,698] Trial 0 finished with value: 0.5 and parameters: {'n_estimators': 650, 'max_depth': 8, 'min_samples_split': 0.8669208084422814, 'min_samples_leaf': 0.258544931121563}. Best is trial 0 with value: 0.5.
[I 2023-08-18 17:08:52,161] Trial 1 finished with value: 0.5 and parameters: {'n_estimators': 150, 'max_depth': 19, 'min_samples_split': 0.764663042959327, 'min_samples_leaf': 0.07191752905955787}. Best is trial 0 with value: 0.5.
[I 2023-08-18 17:09:18,028] Trial 2 finished with value: 0.8334711290696353 and parameters: {'n_estimators': 550, 'max_depth': 20, 'min_samples_split': 0.1840716659403735, 'min_samples_leaf': 0.15737464519376956}. Best is trial 2 with value: 0.8334711290696353.
[I 2023-08-18 17:09:20,050] Trial 3 finished with value: 0.8025650262473274 and parameters: {'n_estimators': 50, 'max_depth': 8, 'min_samples_split': 0.050112183170

ROC AUC: 0.7638537552449831
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.73      0.85     38259
         1.0       0.01      0.79      0.02       146

    accuracy                           0.73     38405
   macro avg       0.51      0.76      0.43     38405
weighted avg       1.00      0.73      0.84     38405



In [55]:
# ADABoosting

# Загрузите свои данные в переменную train_data_copy

# Выделите признаки и целевую переменную
X = train_data_copy.drop(columns=['ID', 'MARKER'])
y = train_data_copy['MARKER']

# Разбейте данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определите целевую функцию для оптимизации гиперпараметров
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 5, 30, step=5)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 1.0, log=True)

    model = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
    return np.mean(scores)

# Создайте объект study для оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Получите оптимальные параметры
best_params = study.best_params

# Создайте модель AdaBoostClassifier с оптимальными параметрами
optimal_model = AdaBoostClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], random_state=42)

# Обучите модель на полной обучающей выборке
optimal_model.fit(X_train, y_train)

# Оцените F1-меру и AUC-ROC Score на тестовой выборке
y_pred = optimal_model.predict(X_test)
test_f1_score = f1_score(y_test, y_pred)
test_auc_score = roc_auc_score(y_test, y_pred)

print("Optimal Hyperparameters:", best_params)
print("Test F1 Score:", test_f1_score)
print("Test AUC-ROC Score:", test_auc_score)


[I 2023-08-18 17:17:44,814] A new study created in memory with name: no-name-0f050778-b8e5-4f31-921c-3f36dd501174
[I 2023-08-18 17:17:48,373] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 15, 'learning_rate': 0.1889200856555388}. Best is trial 0 with value: 0.0.
[I 2023-08-18 17:17:49,590] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 5, 'learning_rate': 0.40377956454702535}. Best is trial 0 with value: 0.0.
[I 2023-08-18 17:17:54,265] Trial 2 finished with value: 0.0 and parameters: {'n_estimators': 20, 'learning_rate': 0.001626343175952954}. Best is trial 0 with value: 0.0.
[I 2023-08-18 17:17:56,663] Trial 3 finished with value: 0.0 and parameters: {'n_estimators': 10, 'learning_rate': 0.04067092448908578}. Best is trial 0 with value: 0.0.
[I 2023-08-18 17:18:03,658] Trial 4 finished with value: 0.0 and parameters: {'n_estimators': 30, 'learning_rate': 0.001757059909770206}. Best is trial 0 with value: 0.0.
[I 2023-08-18 17:18:06,042] Trial 5 

Optimal Hyperparameters: {'n_estimators': 15, 'learning_rate': 0.1889200856555388}
Test F1 Score: 0.0
Test AUC-ROC Score: 0.5
